In [1]:
import cv2

import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision

from PIL import Image

import time

In [2]:
capturing = True
FRAME_RATE = 10
prev = 0

In [3]:
class resNet50_custom(torch.nn.Module):
    def __init__(self):
        super(resNet50_custom, self).__init__()
        self.res50 = torchvision.models.resnet50(pretrained = False, progress = True)
        self.res50.fc = torch.nn.Linear(2048, 1)
    def forward(self, input):
        output = self.res50(input)
        return torch.sigmoid(output)

In [4]:
FaceDetection = resNet50_custom()

FaceDetection.load_state_dict(torch.load('/Users/thierryksstentini/Downloads/dataset/weights_resNet50_FaceDetection_VSC.pth'))
FaceDetection.to('cpu')
FaceDetection.eval();


/Users/thierryksstentini/opt/miniforge3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/thierryksstentini/opt/miniforge3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
def preprocess(img_array):
    preprocessor = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((256,256)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.CenterCrop((224,224)),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225])
            ])
    
    return preprocessor(img_array)
count = 0
vid = cv2.VideoCapture(0)
while(capturing):
    # Capture the video frame by frame
    time_elapsed = time.time() - prev
    ret, frame_instant = vid.read()
    
    if time_elapsed > 1./FRAME_RATE:
        prev = time.time()
        #img_PIL = Image.fromarray(np.uint8(frame_instant)).convert('RGB')
        count += 1
        frame_preprocess = preprocess(frame_instant)
        FaceDetection.eval()
        with torch.no_grad():
            pred = FaceDetection(frame_preprocess.unsqueeze(0))
        print("Prediction en sortie de sigmoid : ", pred)

        if count > 100:
            break




Prediction en sortie de sigmoid :  tensor([[0.4374]])
Prediction en sortie de sigmoid :  tensor([[0.3414]])
Prediction en sortie de sigmoid :  tensor([[0.3094]])
Prediction en sortie de sigmoid :  tensor([[0.3099]])
Prediction en sortie de sigmoid :  tensor([[0.4108]])
Prediction en sortie de sigmoid :  tensor([[0.4433]])
Prediction en sortie de sigmoid :  tensor([[0.4338]])
Prediction en sortie de sigmoid :  tensor([[0.4880]])
Prediction en sortie de sigmoid :  tensor([[0.4900]])
Prediction en sortie de sigmoid :  tensor([[0.4909]])
Prediction en sortie de sigmoid :  tensor([[0.4506]])
Prediction en sortie de sigmoid :  tensor([[0.4256]])
Prediction en sortie de sigmoid :  tensor([[0.4687]])
Prediction en sortie de sigmoid :  tensor([[0.4672]])
Prediction en sortie de sigmoid :  tensor([[0.4025]])
Prediction en sortie de sigmoid :  tensor([[0.5623]])
Prediction en sortie de sigmoid :  tensor([[0.4702]])
Prediction en sortie de sigmoid :  tensor([[0.4489]])
Prediction en sortie de sigm

In [7]:
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

end = time.time()